In [21]:
import re
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from splitter import *
from retriever import *

# Định nghĩa regex pattern cho các tiêu đề (sections)
patterns = [
    r'(?<=\n)Chương \d+: .+',  # Mục lớn như "Chương 1: TỔNG QUAN VỀ BAO BÌ"
    r'(?<=\n)\d+\.\d+ .+',  # Mục nhỏ hơn như "1.1 ĐẶC ĐIỂM CHUNG"
    r'(?<=\n)\d+\.\d+\.\d+ .+',  # Mục nhỏ hơn nữa như "1.1.1 Đặc điểm chung của bao bì"
    r'(?<=\n)\d+\.\d+\.\d+\.\d+ .+'  # Mục nhỏ hơn nữa như "1.1.2.1 Tính năng chứa đựng sản phẩm"
]

# Chia tài liệu thành các đoạn
loaders = TextLoader("doc/"+"spkt.txt", encoding = 'UTF-8')
docs = [loaders.load()]
splitter = PatternSplitter(patterns)
split_docs = splitter.split_documents(docs[0])
# In các đoạn
def beautiful_print(split_docs):
    for split_doc in split_docs:
        print(" ".join([split_doc.metadata['chapter'],split_doc.metadata['section'],split_doc.metadata['subsection'],split_doc.metadata['subsubsection'],]))
        if type(split_doc.metadata['summary']) is str:
            print(split_doc.metadata['summary'][:50])
        else:
            print(split_doc.metadata['summary'])
        print("----------")
        beautiful_print(split_doc.metadata["subtree"])

         
beautiful_print(split_docs)


<class 'list'>
before sum
Limit Token: Hệ thống sẽ chạy lại


In [20]:
from util import *
from retriever import *

question = "đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm B"
en_question = vn_2_en(question)
print(en_question)
retriever = TreeRetriever(documents=split_docs)
result = retriever.invoke(en_question)
print(result.page_content)

    characteristics of packaging from cardboard boxes type group B
Bao bì hộp từ giấy bìa: Là bao bì dạng hộp được sản xuất từ giấy bìa (giấy có định lượng lớn hơn 160 g/m²). Hộp bao gồm một hay nhiều mảnh được cấn và bế. Các đường bế tạo đường viền mảnh hộp, các đường cấn tạo đường nếp để gấp hộp, các mảnh bế sau đó có thể được gấp và dán. Các hộp sau khi gấp dán được lưu trữ và vận chuyển đến khách hàng ở thể phẳng, sau đó được dựng thành hộp trong quá trình đóng gói.
Các mảnh bế là các mảnh phẳng có các đường viền được cắt theo hình dạng trải phẳng của hộp. Các đường phân cách giữa các diện của hộp là các đường cần có nhiệm vụ tạo nếp gấp, các đường này là cơ sở cho việc dựng hộp từ khổ phẳng sang không gian ba chiều.
Do tính dễ vận chuyển, lưu trữ nên các dạng hộp hình hộp chữ nhật được sử dụng rộng rãi trong công nghiệp và cuộc sống. Ngoài bao bì dạng hình hộp chữ nhật, với đặc tính dễ định hình của giấy bìa và khả năng linh hoạt của quá trình cấn bế, và yêu cầu hấp dẫn người tiêu

In [10]:
print(result)

page_content='Xác định các thông tin cần thiết: - Thông tin về sản phẩm:
+ Kích thước và thể tích của sản phẩm bên trong
+ Sản phẩm có cần không gian để dịch chuyển, hay cứng, hoặc có thể nén được
+ Khối lượng sản phẩm
+ Cấp độ của bao bì
+ Số lượng sản phẩm trong bao bì
+ Sản phẩm là chất khí, rắn, chất lỏng, bột hay kem: Nếu là chất khí phải quan tâm đến thể tích, áp suất khối lượng riêng, nhiệt độ. Nếu là chất lỏng cần quan tâm đến độ nhớt, dính, khả năng tạo hơi, khối lượng riêng.
+ Tính chất dễ vỡ của sản phẩm và sự chống va đập cần thiết khi vận chuyển
+ Bị hỏng khi mài mòn, va đập
+ Có bị hư hỏng khi ẩm, có cần chống ẩm hay không
+ Có mùi hay không
+ Khả năng tạo khí
+ Các yêu cầu đặc biệt để bảo quản
+ Có dầu, mỡ...
+ Nhạy cảm với nhiệt độ quá cao hoặc quá thấp
+ Thay đổi màu sắc hay tính chất khi bị chiếu sáng
+ Tính chất hóa học của sản phẩm: tính axít, tính kiềm, hoạt tính hóa học...
- Thông tin về khách hàng
+ Ai là người sử dụng (phụ nữ, đàn ông, độ tuổi, nhóm người nào)
+